In [1]:
### This one is much slower than the Multidim version

from sage.combinat.permutation import Permutations_mset
from sage.combinat.shuffle import SetShuffleProduct
from sage.matrix.args import MatrixArgs_init
from sage.matrix.args import SparseEntry
from sage.combinat.symmetric_group_algebra import e
from itertools import chain

# This function computes the dimension of the cokernel of the last differential of the appropriate line.
# Since the Euler characteristic is known we also obtain the other homology group.
# In order to obtain the multiplicity of the irreducible representation considered, one has to divide the rank
# obtained by the dimension of the representation.
def Compute_Rank(particles, dimension, shape, genus = 0, Show_Progress = False):
    if shape not in Partitions():
        raise(ValueError("The variable 'shape' must be a partition of an integer."))
    line = sum(shape)
    if line > particles:
        raise(ValueError("The variable 'shape' cannot be a partition of an integer larger than 'particles'"))
    
    # initialize the variables
    if genus == 0:
        genus = len(shape)                                     #smallest genus in which the GL irrep is non trivial
    regular = tuples(range(1, genus+1), line)                            #tensor power of H^*
    irrep_VS = (vec*lcm(el.denominator() for el in vec) 
                  for vec in symmetrizer(regular, shape).image().basis())#generator, exhausted by the next variable
    irrep = tuple(dict((regular[i], val) for i, val in vec.items()) 
                  for vec in irrep_VS)                         #summand associated to irrep described by 'shape'
    sym_mult = StandardTableaux(shape).cardinality()                     #dimension of corresponding S_n irrep
    dim_irrep = len(irrep)/sym_mult                                      #dimension of the corresponding GL irrep
    
    eul = sum((-1)^(particles-line+col)*dim_E1(particles, line, col, irrep) for col in range(particles-line+1))
    
    if Show_Progress:
        print(f"Difference of ranks: {eul}") 
    
    # Special case for the top polynomial degree
    if line == particles:
        if Show_Progress:
            actual_line = line*dimension
            print(f"In dimension {actual_line-1}:", 0)
            print(f"In dimension {actual_line}:", eul)
            
        return 0, eul
    
    #General case for the other degrees
    col = particles-line-1 #Verified up to here
    
    domain = E1_irrep(particles, line, col, irrep)
    codomain = E1(particles, line, col+1, regular)
    len_domain = dim_E1(particles, line, col, irrep)
    len_codomain = dim_E1(particles, line, col+1, regular)
    
    mat = diff1_irrep(domain, codomain, len_domain, len_codomain, dimension, Show_Progress = Show_Progress)
    if Show_Progress:
        print("Computing the rank of the differential...", end = "")
    H_top = dim_E1(particles, line, col+1, irrep) - mat.rank()
    if Show_Progress:
        print("......done!")

    H_bottom = H_top - eul
    if Show_Progress:
        actual_line = line*dimension
        print(f"In dimension {actual_line+col} :", H_bottom)
        print(f"In dimension {actual_line+col+1}:", H_top)
        
    return H_bottom, H_top


## Computation for (a basis of) the vector spaces Lie(T) = Lie(k_1) x Lie(k_2) x ... x Lie(k_p)
## The basis is computed by the second function; the first function collates the basis for all T of the same length
## A basis element is represented by a forest whose leaves are labelled by the numbers 1,2,...,n, each of them used
## exactly once; each (rooted) tree corresponding to a Lie word in Lie(k_i).

# returns a tuple
def list_trees(particles, col):
    L = LieAlgebra(QQ, 'x', particles)
    list_gens = [key.to_word()[0] for el in L.gens() for key, _ in el]

    pattern = chain.from_iterable(SetPartitions(list_gens, part) 
                                  for part in Partitions(particles, length = particles-col))
    
    result = chain.from_iterable(cartesian_product([LieOperad(L, part) for part in el]) for el in pattern)
#    result = []
#    for el in pattern:
#        result += cartesian_product([LieOperad(L, part) for part in el])
    return tuple(result)

def LieOperad(LieAlg, variables):
    Basis = LieAlg.Lyndon()
    return tuple(filter(lambda el : all((set(key.to_word()) == variables for key, _ in el)),
                        Basis.graded_basis(len(variables))))
    
###################### Construction of the irreducible representation ####################

# 'regular' is a tensor power, and this function returns a matrix whose image is the invariant subspace under 
# the permutation action encoded by the partition 'shape'
def symmetrizer(regular, shape):
    mat = matrix(QQ, len(regular), sparse = True)
    #mat = [[0 for m_col in range(len(regular))] for m_line in range(len(regular))]
    
    for i, gen in enumerate(regular):
        for tab in StandardTableaux(shape):
            for key, val in e(tab).monomial_coefficients().items():
                mat.add_to_entry(regular.index(tuple(Permutation(key).action(gen))), i, val)
    
    return matrix(QQ, len(regular), len(regular), mat, sparse = True)

## Computation for (a basis of) H^*(X)^k, as well as of the summand for the chosen irreducible representation

# returns a generator
def list_homology(particles, line, col, Basis): #Function verified
    return SetShuffleProduct(([0]*(particles-line-col),), Basis)

def homology_gen(pos_tuple, comb, length): #Function verified
    gen = [0 for i in range(length)]
    for val, index in zip(pos_tuple, comb):
        gen[index] = val
    return tuple(gen)

# returns a generator
def list_homology_irrep(particles, line, col, Basis): #Function verified
    #Since we need the same shuffle for each component of the vector we cannot simply use 'ShuffleProduct'
    basis_el_irrep = lambda comb, basis_el: dict((homology_gen(forest, comb, particles-col), val) 
                                                  for forest, val in basis_el.items())
    return (basis_el_irrep(comb, basis_el) for comb in Combinations(range(particles-col), line)
                                           for basis_el in Basis)

## Computation of the terms on the E1 page

# returns a generator
def E1(particles, line, col, Basis): #Function verified
    return (tuple(zip(tree, gen)) for gen in list_homology(particles, line, col, Basis) 
                                  for tree in list_trees(particles, col))

# returns a generator
def E1_irrep(particles, line, col, Basis): #Function verified
    interleave = lambda tree, gen: dict((tuple(zip(tree, key)), val) for key, val in gen.items())
    return (interleave(tree, gen) for gen in list_homology_irrep(particles, line, col, Basis) 
                                  for tree in list_trees(particles, col))

# returns the dimension of the term E_1^{col,line}
def dim_E1(particles, line, col, Basis): #Function verified
    return stirling_number1(particles, particles-col)*len(Basis)*binomial(particles-col, line)

## Computation of the differential

# returns a matrix; the variables 'domain' and 'codomain' can be a generator
def diff1_irrep(domain, codomain, len_domain, len_codomain, dimension, Show_Progress = False): #Function verified
    #particles, line, col, dimension, Basis, Show_Progress = False):
    images = {el : [] for el in codomain}
#    images = {el : [] for el in E1(particles, line, col+1, Basis, trees)}
    
    if Show_Progress:
        print("Computing differential:...", end = "")
    
    #for i, basis_el in enumerate(E1_irrep(particles, line, col, Basis, trees)):
    for i, basis_el in enumerate(domain):
        for key, val in basis_el.items():
            pos, neg = find_image(key, dimension)
            for p in pos:
                images[p].append((i, val))
            for n in neg:
                images[n].append((i, -val))
            
    if Show_Progress:
        print("...done!")
   
    if Show_Progress:
        print("Computing (sparse) matrix entries:...", end = "")
    
    entries = [SparseEntry(i, index_dom, val) for i, pairs in enumerate(images.values())
                                              for index_dom, val in pairs]
    
    del images
    if Show_Progress:
        print("..................done!")
    
    S = MatrixSpace(ZZ, len_codomain, len_domain, sparse = True)
    return MatrixArgs_init(S, entries).matrix()

def find_image(domain_el, dimension): #Function verified
    pos = []
    neg = []
    
    for left, right in Combinations(range(len(domain_el)), 2):
        pos_temp, neg_temp = cup_product(domain_el, dimension, left, right)
        pos.extend(pos_temp)
        neg.extend(neg_temp)
   
    return pos, neg

def cup_product(domain_el, dimension, left, right): #Function verified
    tree_left, gen_left = domain_el[left]
    tree_right, gen_right = domain_el[right]
    
    if gen_left > 0 and gen_right > 0:
        return [], []
    
    # finds the sign coming from the Koszul rule
    sign = 1
    if gen_left == 0 or dimension%2 == 0: #Koszul sign rule to bring the left-most factor at the beginning
        for _, gen in domain_el[:left]:
            if dimension%2 == 0 or gen == 0:
                sign *= -1
    else: # Koszul sign to swap the tree and the cohomology class
        sign *= -1
    if gen_right == 0 or dimension%2 == 0: #Koszul sign rule to bring the right-most factor at the beginning
        for _, gen in (domain_el[:left]+domain_el[left+1:right]):
            if (dimension%2 == 0 or gen == 0):
                sign *= -1
    
    # computes the bracket of the Lie words and filters the terms according to their sign
    prod = tree_left.bracket(tree_right)

    pos = []
    neg = []
    for key, val in prod.monomial_coefficients().items():
        el, sign_el = add_image(prod.parent()({key:1}),
                                dimension,
                                gen_left+gen_right, #if we get here, at least one of the summands is 0
                                [el for i, el in enumerate(domain_el) if i != left and i != right],
                                sign*val)
        if sign_el > 0:
            pos.append(el)
        else: #if sign_el < 0
            neg.append(el)
        
    return pos, neg 

# Finds the position where to insert the new Lie word and what is the Koszul sign associated to moving the term there
# Maybe evaluating dimension%2 and gen only once would improve performance; would significantly degrade readability
def add_image(key, dimension, gen, result, sign): #Function verified
    pos = 0
    for key_other, gen_other in result:
        if key < key_other:
            break
        pos += 1
        if dimension%2 == 0 or (gen == 0 and gen_other == 0):
            sign *= -1

    result.insert(pos, (key, gen))
    return tuple(result), sign


In [2]:
Compute_Rank(7, 1, [2,1], Show_Progress = True)

Difference of ranks: 196
Computing differential:......done!
Computing (sparse) matrix entries:.....................done!
Computing the rank of the differential.........done!
In dimension 6 : 140
In dimension 7: 336


(140, 336)